# Using a Møller-Plesset-informed initial point

## Introduction

In this tutorial we are going to describe how to use a Møller-Plesset-informed initial point for the Variational Quantum Eigensolver (VQE) with the ground state calculation interface of Qiskit Nature. The goal is to demonstrate that using such an initial point enables us to compute the ground state of an electronic molecular Hamiltonian using fewer evaluations than the default Hartree-Fock initial point. It will also serve to illustrate some changes to how initial points are 

Please see the tutorial on Ground State Solvers to learn the fundamentals of computing the molecular ground state with Qiskit Nature. To know more about the preparation of the Hamiltonian, check out the Electronic structure and Vibrational structure tutorials. 

## The molecule

As usual, the first step is to define the molecular system. In the following, we obtain the electronic part of a hydrogen molecule.

In [ ]:
import qiskit_nature
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver,
    ElectronicStructureDriverType,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

molecule = Molecule(
    geometry=[["Li", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 1.596]]], charge=0, multiplicity=1
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
print(es_problem)

## The ansatz

In [ ]:
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.properties.second_quantization.electronic import ParticleNumber

qiskit_nature.settings.dict_aux_operators = True

# generate the second-quantized operators
second_q_ops = es_problem.second_q_ops()

grouped_property = es_problem.grouped_property_transformed

particle_number = grouped_property.get_property(ParticleNumber)
num_particles = (particle_number.num_alpha, particle_number.num_beta)
num_spin_orbitals = particle_number.num_spin_orbitals

# setup the qubit converter
qubit_converter = QubitConverter(JordanWignerMapper())

main_op = second_q_ops["ElectronicEnergy"]
# map to qubit operators
qubit_op = qubit_converter.convert(main_op, num_particles=num_particles)

# setup the ansatz for VQE
ansatz = UCCSD(
    qubit_converter=qubit_converter,
    num_particles=num_particles,
    num_spin_orbitals=num_spin_orbitals,
)

## The initial point

In [ ]:
from qiskit_nature.algorithms.initial_points import HFInitialPoint

hf_initial_point = HFInitialPoint()
hf_initial_point.grouped_property = grouped_property
hf_initial_point.ansatz = ansatz
hf_initial_point_np = hf_initial_point.to_numpy_array()

In [ ]:
from qiskit_nature.algorithms.initial_points import MP2InitialPoint

mp2_initial_point = MP2InitialPoint()
mp2_initial_point.grouped_property = grouped_property
mp2_initial_point.ansatz = ansatz
mp2_initial_point_np = mp2_initial_point.to_numpy_array()

## The Solver

Then we need to define a solver. The solver is the algorithm through which the ground state is computed.

To define the VQE solver one needs two essential elements:

1. A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance [Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected.
2. An initial state: the initial state of the qubits. In the factory used above, the qubits are initialized in the Hartree-Fock (see the electronic structure tutorial) initial state (the qubits corresponding to occupied MOs are $|1\rangle$ and those corresponding to virtual MOs are $|0\rangle$.
3. The backend: this is the quantum machine on which the right part of the figure above will be performed. Here we ask for the perfect quantum emulator (```aer_simulator_statevector```). 

One could also use any available ansatz / initial state or even define one's own. For instance,



We will be using the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithms works by exchanging information between a classical and a quantum computer as depicted in the following figure.

<img src="aux_files/vqe.png" width="600">

Let's initialize a VQE solver.

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import L_BFGS_B
from qiskit.algorithms import VQE

optimizer = L_BFGS_B()

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
hf_vqe = VQE(
    ansatz, optimizer=optimizer, quantum_instance=quantum_instance, initial_point=hf_initial_point_np
)

Let's initialize a VQE solver using a MPInitialPoint.

In [ ]:
vqe_mp2 = VQE(
    ansatz, optimizer=optimizer, quantum_instance=quantum_instance, initial_point=mp2_initial_point_np
)

## The calculation and results

We are now ready to run the calculation.

In [ ]:
from qiskit_nature.algorithms import GroundStateEigensolver

hf_gse = GroundStateEigensolver(qubit_converter, hf_vqe)
hf_result = hf_gse.solve(es_problem)
print(hf_result)

In [ ]:
mp2_gse = GroundStateEigensolver(qubit_converter, vqe_mp2)
mp2_result = mp2_gse.solve(es_problem)
print(mp2_result)